In [1]:
import requests
import geopandas as gpd

bbox = [-105.3, 37.0, -104.5, 37.6]  # Example BBOX
response = requests.post("http://localhost:5001/api/fetch_trails", json={"bbox": bbox})

if response.status_code == 200:
    fetched_trails = gpd.read_file(response.text)
    print(fetched_trails.head())
else:
    print("Error:", response.text)

DataSourceError: {  "redirect": "/selections"}: No such file or directory

In [33]:
pd.set_option('display.max_columns', None)  # Show all columns

In [7]:
DATA_RAW_PATH = "data/raw"
os.makedirs(DATA_RAW_PATH, exist_ok=True)

# Define USFS Feature Server URLs
USFS_TRAILS_URL = "https://apps.fs.usda.gov/arcx/rest/services/EDW/EDW_TrailNFSPublish_01/MapServer/0"
# USFS_TRAILHEADS_URL = "https://apps.fs.usda.gov/arcx/rest/services/EDW/EDW_Trailheads_01/MapServer/0"

# Initialize ArcGIS GIS object (No Authentication Needed for Public Data)
gis = GIS()

In [13]:
def gdf_from_feature_layer(feature_layer, wkid):
        if feature_layer.features:
            geojson = feature_layer.to_geojson
            gdf = gpd.read_file(geojson)
            # if gdf.geometry is None or gdf.geometry.is_empty.all():
            #     fixed_geojson = self._correct_multipolygon_nesting_as_string(
            #         geojson)
            #     gdf = gpd.read_file(fixed_geojson)
            gdf = gdf.to_crs(epsg=wkid)
            return gdf
        else:
            return gpd.GeoDataFrame()

In [18]:
def get_feature_layer_from_server(layer_url, bbox):
    """Fetch features from an Esri Feature Server using a bounding box."""
    feature_layer = FeatureLayer(layer_url, gis)
    wkid = 4326  # Ensure projection is in WGS84
    
    try:
        # Convert bbox to Esri's dictionary format
        bbox_dict = {
            "xmin": bbox[0], "ymin": bbox[1], "xmax": bbox[2], "ymax": bbox[3],
            "spatialReference": {"wkid": wkid}
        }
        
        print(bbox_dict)
        # Query Feature Layer
        query_filter = intersects(bbox_dict, sr=wkid)
        features = feature_layer.query(geometry_filter=query_filter, out_sr=wkid, out_fields="*")
        
        # Convert to GeoDataFrame
        gdf = gdf_from_feature_layer(features, wkid)
        if gdf.empty:
            print(f"No features found for {layer_url}")
            return None

        # Ensure CRS is in WGS84
        if gdf.crs is None or gdf.crs != "EPSG:4326":
            gdf = gdf.to_crs(epsg=4326)

        return gdf

    except Exception as e:
        print(f"Error fetching layer {layer_url}: {e}")
        return None

In [19]:
def fetch_usfs_data(bbox):
    """Fetch USFS trails and trailheads within the user's bounding box."""
    print("Fetching USFS Trails...")
    trails_gdf = get_feature_layer_from_server(USFS_TRAILS_URL, bbox)

    # print("Fetching USFS Trailheads...")
    # trailheads_gdf = get_feature_layer_from_server(USFS_TRAILHEADS_URL, bbox)

    # Save raw results for further processing
    if trails_gdf is not None:
        trails_gdf.to_file(f"{DATA_RAW_PATH}/usfs_trails.geojson", driver="GeoJSON")
    # if trailheads_gdf is not None:
    #     trailheads_gdf.to_file(f"{DATA_RAW_PATH}/usfs_trailheads.geojson", driver="GeoJSON")

    return trails_gdf

In [20]:
bbox = [-105.14490332783127, 37.03295622962676, -104.56681540041795, 37.489198771684514]

In [23]:
gdf = fetch_usfs_data(bbox)

Fetching USFS Trails...
{'xmin': -105.14490332783127, 'ymin': 37.03295622962676, 'xmax': -104.56681540041795, 'ymax': 37.489198771684514, 'spatialReference': {'wkid': 4326}}


In [34]:
gdf.head()

,TRAIL_NO,TRAIL_NAME,TRAIL_TYPE,TRAIL_CN,BMP,EMP,SEGMENT_LENGTH,ADMIN_ORG,MANAGING_ORG,SECURITY_ID,ATTRIBUTESUBSET,NATIONAL_TRAIL_DESIGNATION,TRAIL_CLASS,TERRA_BASE_SYMBOLOGY,ACCESSIBILITY_STATUS,TRAIL_SURFACE,SURFACE_FIRMNESS,TYPICAL_TRAIL_GRADE,TYPICAL_TREAD_WIDTH,MINIMUM_TRAIL_WIDTH,TYPICAL_TREAD_CROSS_SLOPE,SPECIAL_MGMT_AREA,MVUM_SYMBOL,ALLOWED_TERRA_USE,ALLOWED_SNOW_USE,HIKER_PEDESTRIAN_MANAGED,HIKER_PEDESTRIAN_ACCPT_DISC,HIKER_PEDESTRIAN_RESTRICTED,PACK_SADDLE_MANAGED,PACK_SADDLE_ACCPT_DISC,PACK_SADDLE_RESTRICTED,BICYCLE_MANAGED,BICYCLE_ACCPT_DISC,BICYCLE_RESTRICTED,MOTORCYCLE_MANAGED,MOTORCYCLE_ACCPT_DISC,MOTORCYCLE_RESTRICTED,ATV_MANAGED,ATV_ACCPT_DISC,ATV_RESTRICTED,FOURWD_MANAGED,FOURWD_ACCPT_DISC,FOURWD_RESTRICTED,SNOWMOBILE_MANAGED,SNOWMOBILE_ACCPT_DISC,SNOWMOBILE_RESTRICTED,SNOWSHOE_MANAGED,SNOWSHOE_ACCPT_DISC,SNOWSHOE_RESTRICTED,XCOUNTRY_SKI_MANAGED,XCOUNTRY_SKI_ACCPT_DISC,XCOUNTRY_SKI_RESTRICTED,MOTOR_WATERCRAFT_MANAGED,MOTOR_WATERCRAFT_ACCPT_DISC,MOTOR_WATERCRAFT_RESTRICTED,NONMOTOR_WATERCRAFT_MANAGED,NONMOTOR_WATERCRAFT_ACCPT_DISC,NONMOTOR_WATERCRAFT_RESTRICTED,GIS_MILES,TERRA_MOTORIZED,SNOW_MOTORIZED,WATER_MOTORIZED,OBJECTID,SHAPE.LEN,HIKER_PEDESTRIAN_ACCPT,HIKER_PEDESTRIAN_DISC,PACK_SADDLE_ACCPT,PACK_SADDLE_DISC,BICYCLE_ACCPT,BICYCLE_DISC,MOTORCYCLE_ACCPT,MOTORCYCLE_DISC,ATV_ACCPT,ATV_DISC,FOURWD_ACCPT,FOURWD_DISC,SNOWCOACH_SNOWCAT_MANAGED,SNOWCOACH_SNOWCAT_ACCPT,SNOWCOACH_SNOWCAT_DISC,SNOWCOACH_SNOWCAT_ACCPT_DISC,SNOWCOACH_SNOWCAT_RESTRICTED,SNOWMOBILE_ACCPT,SNOWMOBILE_DISC,SNOWSHOE_ACCPT,SNOWSHOE_DISC,XCOUNTRY_SKI_ACCPT,XCOUNTRY_SKI_DISC,MOTOR_WATERCRAFT_ACCPT,MOTOR_WATERCRAFT_DISC,NONMOTOR_WATERCRAFT_ACCPT,NONMOTOR_WATERCRAFT_DISC,E_BIKE_CLASS1_MANAGED,E_BIKE_CLASS1_ACCPT,E_BIKE_CLASS1_DISC,E_BIKE_CLASS1_RESTRICTED,E_BIKE_CLASS2_MANAGED,E_BIKE_CLASS2_ACCPT,E_BIKE_CLASS2_DISC,E_BIKE_CLASS2_RESTRICTED,E_BIKE_CLASS3_MANAGED,E_BIKE_CLASS3_ACCPT,E_BIKE_CLASS3_DISC,E_BIKE_CLASS3_RESTRICTED,GLOBALID,geometry
0,1309,NORTH FORK,TERRA,6818010465,1.3600,1.9300,0.5700,021203,021203,0212,TrailNFS_MGMT,1,3,TC3,None,NATIVE MATERIAL,None,12-20%,TW03 - 18-24 INCHES,None,None,None,NaN,321,None,None,01/01-12/31,None,01/01-12/31,None,None,None,01/01-12/31,None,None,None,01/01-12/31,None,None,01/01-12/31,None,None,01/01-12/31,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.592,N,None,None,101462,0.008688,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,{1EC41432-3CB0-4CE9-B591-F8E0D0FFBCAD},"MULTILINESTRING ((-105.1205 37.27029, -105.120..."
1,1309,NORTH FORK,TERRA,6818010465,0.0000,0.2375,0.2375,021203,021203,0212,TrailNFS_MGMT,1,3,TC3,None,NATIVE MATERIAL,None,12-20%,TW03 - 18-24 INCHES,None,None,None,NaN,321,None,None,01/01-12/31,None,01/01-12/31,None,None,None,01/01-12/31,None,None,None,01/01-12/31,None,None,01/01-12/31,None,None,01/01-12/31,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,0.246,N,None,None,102097,0.003630,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,{513BD06E-60E6-440B-B559-E039C7ECA920},"MULTILINESTRING ((-105.10957 37.25502, -105.10..."
2,1300.A,SHORTCUT,TERRA,9497010465,0.0000,1.2900,1.2900,021203,021203,0212,TrailNFS_MGMT,1,3,TC3,None,NATIVE MATERIAL,None,12-20%,TW03 - 18-24 INCHES,None,None,None,7.0,54321,None,None,01/01-12/31,None,None,None,None,None,01/01-12/31,None,None,01/01-12/31,None,01/01-12/31,None,None,None,None,01/01-12/31,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1.313,Y,None,None,102602,0.020547,None,None,None,None,None,None,01/01-12/31,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,01/01-12/31,None,None,None,01/01-12/31,None,None,None,01/01-12/31,None,N

In [28]:
gdf.columns

Index(['TRAIL_NO', 'TRAIL_NAME', 'TRAIL_TYPE', 'TRAIL_CN', 'BMP', 'EMP',
       'SEGMENT_LENGTH', 'ADMIN_ORG', 'MANAGING_ORG', 'SECURITY_ID',
       ...
       'E_BIKE_CLASS2_MANAGED', 'E_BIKE_CLASS2_ACCPT', 'E_BIKE_CLASS2_DISC',
       'E_BIKE_CLASS2_RESTRICTED', 'E_BIKE_CLASS3_MANAGED',
       'E_BIKE_CLASS3_ACCPT', 'E_BIKE_CLASS3_DISC', 'E_BIKE_CLASS3_RESTRICTED',
       'GLOBALID', 'geometry'],
      dtype='object', length=105)

In [26]:
gdf.explore()